In [ ]:
from pyspark.mllib.linalg import SparseVector
from pyspark.mllib.linalg.distributed import RowMatrix
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import time
from collections import defaultdict
from pyspark.sql import functions as sfunc
from pyspark.sql import types as stypes
import math
import sys
from pyspark.ml.linalg import SparseVector
from pyspark.mllib.linalg.distributed import RowMatrix
from operator import itemgetter


In [ ]:
schema = stypes.StructType().add("fv", stypes.StringType()).add("sku", stypes.StringType()).add("score", stypes.FloatType())
train_df = spark.read.csv('gs://lbanor/pyspark/train_query*.gz', header=True, schema=schema)
train_df.createOrReplaceTempView('test1')

In [ ]:
query = """
SELECT
  sku,
  ROW_NUMBER() OVER (ORDER BY SUM(1)) -1 idx
FROM test1
GROUP BY 1
"""
skus_rdd = spark.sql(query).rdd

In [ ]:
d = {row.sku: row.idx for row in skus_rdd.collect()}
db = sc.broadcast(d)

id_ = {value: key for key, value in d.items()}
id_b = sc.broadcast(id_)

In [ ]:
query_users_items = """
SELECT
data
FROM(
  SELECT
    fv,
    COLLECT_LIST(STRUCT(sku, score * 2 AS score)) data
  FROM test1
  GROUP BY 1
)
WHERE size(data) between 2 and 20
LIMIT 3
"""

In [ ]:
users_rdd = spark.sql(query_users_items).rdd

In [ ]:
def make_sparse(row):
    tmp = sorted([(db.value[i.sku], i.score) for i in row.data], key=itemgetter(0))
    return (SparseVector(len(db.value), [e[0] for e in tmp], [e[1] for e in tmp]),)

t0 = time.time()
mat = RowMatrix(users_rdd.map(lambda x: make_sparse(x)).toDF())

In [ ]:
a = mat.columnSimilarities(0)